In [38]:
from gekko import GEKKO
import numpy as np
import matplotlib.pyplot as plt
import time

m = GEKKO(remote = False)


#time horizon: 6h
Thor = 5*60 #sec
Ts = 1 #sec
m.time = np.linspace(0,int(Thor-1),int(Thor/Ts))
n = Ts*Thor

np.random.seed(1)
# External Disturbances
Qin_hat = np.random.randint(0,200,size=n)

# CO2 Cost
co2 = np.random.randint(70,100,size=n)

# Limits
h_low = 120*np.ones(n)
h_upper = 180*np.ones(n)


# Controlled Variable
# Inital value and Boundaries
h = m.CV(value=150)  # min 120, max 180, init: last value
Qout = m.CV()
P = m.CV()

# Manipulated Variable
#w = m.MV(value=0, lb=0, ub=1500, integer=True)   # min 0, max 1500, init: last value
w = m.MV()
#w.DMAX =       # Max derivative
#w.DCOST = 20    # Cost for movement


# A (na x ny)
A_power = np.array([[3.7971E-01],\
                    [5.1151E-02],\
                    [3.1656E-01]])
A_outflow = np.array([[1.2995],\
                    [-4.3918E-01],\
                    [4.5231E-02],\
                    [0],\
                    [1.0387E-02]])

# B (ny x (nb x nu))
B_power = np.array([[[5.5404E-02],[-1.4118E-02],[-2.3452E-02]]]).T
B_outflow = np.array([[[7.1049E-02],[4.1327E-02],[6.4801E-02],[0],[-6.6199E-02]]]).T

C_power = np.array([-6.6831])
C_outflow = np.array([-7.3292E+01])

p_power = {'a':A_power,'b':B_power,'c':C_power}
p_outflow = {'a':A_outflow,'b':B_outflow,'c':C_outflow}

m.arx(p_power,P,w)
m.arx(p_outflow,Qout,w)

# Equations and paramters
area = 18 # m²
hL = m.Param(value=h_low[0]) 
hH = m.Param(value=h_upper[0])
c = m.Param(value=co2[0])


w.STATUS = 1

# Lower and Upper Limits
eH = m.CV(value=0)
eL = m.CV(value=0)

eH.SPHI=0         #Set point high for linear error model.
eH.WSPHI=100        #Objective function weight on upper set point for linear error model.
eH.WSPLO=0        # Objective function weight on lower set point for linear error model
eH.STATUS =1      # eH : Error is considered in the objective function.

eL.SPLO=0
eL.WSPHI=0
eL.WSPLO=1000
eL.STATUS = 1   

m.Equation(h.dt() == 1/area *(Qin_hat[0] - Qout)/3600*100)
m.Equations([eH==h-hH,eL==h-eL]) # Errors

# Objective
m.Minimize(c*P)


# steady state initialization
#m.options.IMODE = 1
#m.solve(disp=True,debug=True)



In [39]:

hL.value = h_low
hH.value = h_upper
c.VALUE = co2

#Set Up MPC
m.options.IMODE = 6 # MPC mode in Gekko.
m.options.NODES = 2  # Collocation nodes.
m.options.SOLVER = 1

m.solve()
#m.solve()

# last_co2_fetch = 0

# Should be inf.
# loops = 100
# for i in range(1,loops):
#     Loop that executes every XX sec. actual control loop
#     time.sleep(10)


#     every 15min fetch new CO2 data
#     if (time.time() - last_co2_fetch > 15*60):
#         fetch co2 data for the next 6h and save it
#         np.random.randint(50,200,size=xxx)
#         save to array

#         last_co2_fetch = time.time()

#     Solve MPC
#     m.solve()    
#     m.solve()


#     New Setpoints to Pump

 ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            2
   Constants    :            0
   Variables    :            9
   Intermediates:            0
   Connections  :            4
   Equations    :            4
   Residuals    :            4
 
 Number of state variables:          23322
 Number of total equations: -        23023
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :            299
 
 ----------------------------------------------
 Dynamic Control with APOPT Solver
 ----------------------------------------------
 
 Iter    Objective  Convergence
    0  1.54772E+06  1.49000E+02
    1 -3.60148E+11  1.41415E+00
    2 -1.02762E+15  9.20867E-01
    3 -1.02762E+15  6.52671E-09
 Unbounded soluti

Exception: @error: Solution Not Found
